In [17]:
import requests
import time
import uuid

from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#from selenium.webdriver.chrome.options import Options
#from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome()

class scraper:
    def intitialize(self, url, search_term):
        global_ids = scraper.getUniqueID(scraper, 'https://www.pickuplimes.com/recipe/spicy-garlic-wok-noodles-213')
        self.getURL(url)
        #self.getTitle()
        #self.acceptCookies()
        #self.getAllRecipePages()
        #self.getSourceCode()
        #self.search(search_term)
        #time.sleep(3)
        #self.home()
        #self.findRecipeList()
        #self.getRecipeDetails()
        #self.getRecipes()
        #self.getPageURL()
        #self.getUniqueID()
        #self.getRecipeDetails()
        #self.getRecipeDetails(self)
        #time.sleep(3)
        self.closeSession()

    def getURL(url):
        driver.get(url) 

    def getTitle():
        print(driver.title)

    def closeSession():
        time.sleep(3)
        driver.quit()

    def getSourceCode():
        print(driver.page_source)
    
    def search(search_term):
        try:
            button = driver.find_element(By.ID, 'nav-searchbar-btn')
            button.click()
            try:
                search_bar = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "sb")))
                try:
                    search_bar.send_keys(search_term)
                    search_bar.send_keys(Keys.RETURN) # Return = Enter
                except:
                    print("Exception: No search term input")
            except:
                print("Exception: No search bar found")
        except:
            print("Exception: No search button found")

    def home():
        title = driver.find_element(By.ID, 'nav-image')
        title.click()

    def findRecipeList():
        button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.LINK_TEXT, 'Recipes')))
        button.click()

    def acceptCookies():
        try:
            cookie_button = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]')
            cookie_button.click()
            print("Removed Cookies")
        except:
            print("Exeption: Didnt Find Cookie Button")
    
    def getRecipes():
        main = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'index-item-container')))      
        print("Found results")
        articles = main.find_elements(By.TAG_NAME, 'li')
        print("Number of Recipes:", len(articles))

        for i in articles:
            print("Recipe:" , i.text)

    def getPageURL():
        print("URL:", driver.current_url)

    def getAllRecipePages(self):
        pages = []
        self.getURL('https://www.pickuplimes.com/')
        scraper.findRecipeList()
        page = [driver.current_url]

        #total_pages = driver.find_element(By.CLASS_NAME, 'page-text')

        total_pages = [1, 2, 3, 4, 5]

        for i in total_pages:
            current_page = driver.current_url
            url_change = "?page=" + str(i)
            next_page = current_page + url_change
            print(next_page)
            pages.append(next_page)
            print("Number of Pages:", len(pages))

    def getUniqueID(self, url):
        
        page_ID = url
        just_ID = page_ID.replace(str("https://www.pickuplimes.com/recipe/"), "")

        ids = (just_ID, uuid.uuid4())

        return ids

    def getRecipeDetails(self):
        self.getURL('https://www.pickuplimes.com/recipe/spicy-garlic-wok-noodles-213')

        name = driver.find_element(By.XPATH, '//*[@id="header-info-col"]/div/header/h1')
        tag = driver.find_element(By.XPATH, '//*[@id="header-info-col"]/div/header/a[1]/div/p')
        description = driver.find_element(By.XPATH, '//*[@id="header-info-col"]/div/header/span') 
        time_total = driver.find_element(By.XPATH, '//*[@id="recipe-info-container"]/div[2]')
        time_prep = driver.find_element(By.XPATH, '//*[@id="recipe-info-container"]/div[3]')
        time_cook = driver.find_element(By.XPATH, '//*[@id="recipe-info-container"]/div[4]')
        allergens = driver.find_element(By.XPATH, '//*[@id="allergen-info-container"]/div[1]/div') 
        swap = driver.find_element(By.XPATH, '//*[@id="allergen-info-container"]/div[2]/div')
        free_from = driver.find_element(By.XPATH, '//*[@id="allergen-info-container"]/div[3]/div')  
        ingredients = driver.find_element(By.XPATH, '//*[@id="ingredient-direction-container"]/div/div[2]')
        directions = driver.find_element(By.XPATH, '//*[@id="ingredient-direction-container"]/div/div[4]/section/ol')
        notes = driver.find_element(By.XPATH, '//*[@id="ingredient-direction-container"]/div/div[4]/section/ul[1]/li')
        storage = driver.find_element(By.XPATH, '//*[@id="ingredient-direction-container"]/div/div[4]/section/ul[2]/li')
        nutrition = driver.find_element(By.XPATH, '//*[@id="nut-info"]')

        picture_main = driver.find_element(By.XPATH, '//*[@id="main-image-container"]/img')

        #print("Recipe:", name.text)
        #print("Type:", tag.text)
        #print("Description:", description.text)
        #print(time_total.text)
        #print(time_prep.text)
        #print(time_cook.text)
        #print("Allergens:", allergens.text)
        #print("Swap:", swap.text)
        #print("Free from:", free_from.text)
        #print(ingredients.text)
        #print(directions.text)
        #print("Notes:", notes.text)
        #print("Storage:", storage.text)

        self.closeSession()

scraper.intitialize(scraper, 'https://www.pickuplimes.com', 'lemons')
#global_ids = scraper.getUniqueID(scraper, 'https://www.pickuplimes.com/recipe/spicy-garlic-wok-noodles-213')
#print(global_ids)
#scraper.closeSession()

# IDEALS
# store all recipe images in a list
# link getTitle() and home()
# link getPageURL() and getUniqueID()
# get search results
# get search results on more than the first page
# counts how many recipes there are
# gets recipe info and puts it in a dictionary
# bypass login
# store all recipe unique id in a list
# replace all XPaths
# a method that makes a list containing all buttons and cycles through them looking for a link_text given as an argument

('spicy-garlic-wok-noodles-213', UUID('edf29886-dc92-4fd5-b8a4-296d5fb481a2'))


In [176]:
# ZOOPLA
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

driver = webdriver.Chrome()  
url = "https://zoopla.co.uk" 
driver.get(url)

time.sleep(5) # Slow it down otherwise it will miss the button 

try: 
    frame_id = "gdpr-consent-notice"
    driver.switch_to.frame(frame_id)
    print("Switched to frame")
    cookie_button = driver.find_element(By.XPATH, '//*[@id="save"]')
    cookie_button.click()
    print("Accepted Cookies")
except:
    print("No Cookie Button")
    pass

url = "https://www.zoopla.co.uk/for-sale/property/london/?q=london&results_sort=newest_listings&search_source=home"
driver.get(url)

property = driver.find_element(By.XPATH, '//*[@id="listing_61207984"]/div[1]/div[2]')
a_tag = property.find_element(By.TAG_NAME, 'a')
link = a_tag.get_attribute('href')
driver.get(link)
print(link)

price = driver.find_element(By.XPATH, '//span[@data-testid="price"]').text 
address = driver.find_element(By.XPATH, '//span[@data-testid="address-label"]').text 
bedrooms = driver.find_element(By.XPATH, '//span[@data-testid="beds-label"]').text  

div_tag = driver.find_element(By.XPATH, '//div[@data-testid="truncated_text_container"]') 
span_tag = div_tag.find_element(By.XPATH,'.//span') 
description = span_tag.text 

dict_properties = {'Price': [], 'Address': [], 'Bedrooms': [], 'Description': []} 
dict_properties['Price'].append(price) 
dict_properties['Address'].append(address) 
dict_properties['Bedrooms'].append(bedrooms) 
dict_properties['Description'].append(description) 
dict_properties
print(dict_properties)

time.sleep(3)
driver.quit()

Switched to frame
Accepted Cookies
https://www.zoopla.co.uk/for-sale/details/61207984/?search_identifier=c14ac08bf282507f6273830c65fcde12
{'Price': ['£475,000'], 'Address': ['Langdale Road, Thornton Heath, Surrey CR7'], 'Bedrooms': ['3 beds'], 'Description': ['This mid-terrace house, which is in need of some modernisation, would make an ideal family home.\n\nThe ground floor comprises two reception rooms, kitchen, and a lean too with access to the private rear garden from both the kitchen and the rear reception.\n\nUpstairs has two double bedrooms and a good-sized single bedroom as well as a second bathroom with three-piece suite.\n\nThe property is situated within walking distance to local amenities and schools as well as Thornton Heath Station.']}


In [20]:
my_list = [1, 2, 3]
print(my_list * 2)


[1, 2, 3, 1, 2, 3]
